In [1]:
def entropy(class_probabilities):
    return sum(-p * math.log(p, 2) 
               for p in class_probabilities 
               if p)

In [2]:
def class_probabilities(labels):
    total_count = len(labels)
    return [count / total_count
            for count in Counter(labels).values()]

In [3]:
def data_entropy(labeled_data):
    labels = [label for _, label in labeled_data]
    probabilities = class_probabilities(labels)
    return entropy(probabilities)

In [4]:
def partition_entropy(subsets):
    total_count = sum(len(subset) for subset in subsets)

    return sum( data_entropy(subset) * len(subset) / total_count
                for subset in subsets )

In [5]:
inputs = [
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'no'},   False),
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'yes'},  False),
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'no'},     True),
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'no'},  True),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'yes'},    False),
        ({'level':'Mid','lang':'R','tweets':'yes','phd':'yes'},        True),
        ({'level':'Senior','lang':'Python','tweets':'no','phd':'no'}, False),
        ({'level':'Senior','lang':'R','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'Python','tweets':'yes','phd':'no'}, True),
        ({'level':'Senior','lang':'Python','tweets':'yes','phd':'yes'},True),
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'yes'},    True),
        ({'level':'Mid','lang':'Java','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'yes'},False)
    ]

In [19]:
from collections import Counter, defaultdict
from functools import partial
import math, random

def partition_by(inputs, attribute):
    groups = defaultdict(list)
    for input in inputs:
        key = input[0][attribute]
        groups[key].append(input)
    return groups

In [20]:
def partition_entropy_by(inputs,attribute):
    partitions = partition_by(inputs, attribute)
    return partition_entropy(partitions.values())

In [24]:
def classify(tree, input):

    if tree in [True, False]:
        return tree

    attribute, subtree_dict = tree
    subtree_key = input.get(attribute)  

    if subtree_key not in subtree_dict: 
        subtree_key = None              

    subtree = subtree_dict[subtree_key] 
    return classify(subtree, input)     

In [26]:
def build_tree_id3(inputs, split_candidates=None):

    if split_candidates is None:
        split_candidates = inputs[0][0].keys()

    num_inputs = len(inputs)
    num_trues = len([label for item, label in inputs if label])
    num_falses = num_inputs - num_trues

    if num_trues == 0: return False                    
    if num_falses == 0: return True                     
    if not split_candidates: return num_trues >= num_falses  

    best_attribute = min(split_candidates,
        key=partial(partition_entropy_by, inputs))

    partitions = partition_by(inputs, best_attribute)
    new_candidates = [a for a in split_candidates
                      if a != best_attribute]

    subtrees = { attribute : build_tree_id3(subset, new_candidates)
                 for attribute, subset in partitions.items() }

    subtrees[None] = num_trues > num_falses

    return (best_attribute, subtrees)

In [29]:
classify(build_tree_id3(inputs), 
         { "level" : "Junior",
          "lang" : "Java",
          "tweets" : "yes",
          "phd" : "no"})

True

In [32]:
classify(build_tree_id3(inputs), 
         { "level" : "Junior",
          "lang" : "Java",
          "tweets" : "yes",
          "phd" : "yes"})

False

In [33]:
tree = build_tree_id3(inputs)
tree

('level',
 {'Senior': ('tweets', {'no': False, 'yes': True, None: False}),
  'Mid': True,
  'Junior': ('phd', {'no': True, 'yes': False, None: True}),
  None: True})

In [34]:
inputs20 = [
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'no'},   False),
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'yes'},  False),
        ({'level':'Senior','lang':'Java','tweets':'yes','phd':'no'},   True),
        ({'level':'Senior','lang':'Java','tweets':'yes','phd':'yes'},  True),
    
        ({'level':'Senior','lang':'Python','tweets':'no','phd':'no'},  False),
        ({'level':'Senior','lang':'Python','tweets':'no','phd':'yes'}, False),
        ({'level':'Senior','lang':'Python','tweets':'yes','phd':'no'}, True),
        ({'level':'Senior','lang':'Python','tweets':'yes','phd':'yes'},True),
    
        ({'level':'Senior','lang':'R','tweets':'no','phd':'no'},  True),
        ({'level':'Senior','lang':'R','tweets':'no','phd':'yes'}, False),
        ({'level':'Senior','lang':'R','tweets':'yes','phd':'no'}, False),
        ({'level':'Senior','lang':'R','tweets':'yes','phd':'yes'},True),
        
    
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'no'},     True),        
        ({'level':'Mid','lang':'R','tweets':'yes','phd':'yes'},        True),
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'yes'},    True),
        ({'level':'Mid','lang':'Java','tweets':'yes','phd':'no'},      True),
    
        ({'level':'Junior','lang':'Python','tweets':'yes','phd':'no'}, True),
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'yes'},False),
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'no'},  True),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'no'},      True),
    ]

In [35]:
tree = build_tree_id3(inputs20)
tree

('tweets',
 {'no': ('level',
   {'Senior': ('lang',
     {'Java': False,
      'Python': False,
      'R': ('phd', {'no': True, 'yes': False, None: False}),
      None: False}),
    'Mid': True,
    'Junior': ('phd', {'yes': False, 'no': True, None: False}),
    None: False}),
  'yes': ('lang',
   {'Java': True,
    'Python': True,
    'R': ('level',
     {'Senior': ('phd', {'no': False, 'yes': True, None: False}),
      'Mid': True,
      'Junior': True,
      None: True}),
    None: True}),
  None: True})

In [45]:
inputs50 = [
        #Senior
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'no'},   False),
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'yes'},  False),
        ({'level':'Senior','lang':'Java','tweets':'yes','phd':'no'},   True),
        ({'level':'Senior','lang':'Java','tweets':'yes','phd':'yes'},  True),
    
        ({'level':'Senior','lang':'Python','tweets':'no','phd':'no'},  False),
        ({'level':'Senior','lang':'Python','tweets':'no','phd':'yes'}, False),
        ({'level':'Senior','lang':'Python','tweets':'yes','phd':'no'}, True),
        ({'level':'Senior','lang':'Python','tweets':'yes','phd':'yes'},True),
    
        ({'level':'Senior','lang':'R','tweets':'no','phd':'no'},  True),
        ({'level':'Senior','lang':'R','tweets':'no','phd':'yes'}, False),
        ({'level':'Senior','lang':'R','tweets':'yes','phd':'no'}, False),
        ({'level':'Senior','lang':'R','tweets':'yes','phd':'yes'},True),
    
        ({'level':'Senior','lang':'C#','tweets':'no','phd':'no'},   False),
        ({'level':'Senior','lang':'C#','tweets':'no','phd':'yes'},  False),
        ({'level':'Senior','lang':'C#','tweets':'yes','phd':'no'},   False),
        ({'level':'Senior','lang':'C#','tweets':'yes','phd':'yes'},  True),
        
        ##Mid
        ({'level':'Mid','lang':'Java','tweets':'no','phd':'no'},   False),
        ({'level':'Mid','lang':'Java','tweets':'no','phd':'yes'},  True),
        ({'level':'Mid','lang':'Java','tweets':'yes','phd':'no'},   True),
        ({'level':'Mid','lang':'Java','tweets':'yes','phd':'yes'},  True),
    
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'no'},  False),
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'yes'}, True),
        ({'level':'Mid','lang':'Python','tweets':'yes','phd':'no'}, False),
        ({'level':'Mid','lang':'Python','tweets':'yes','phd':'yes'},True),
    
        ({'level':'Mid','lang':'R','tweets':'no','phd':'no'},  True),
        ({'level':'Mid','lang':'R','tweets':'no','phd':'yes'}, True),
        ({'level':'Mid','lang':'R','tweets':'yes','phd':'no'}, False),
        ({'level':'Mid','lang':'R','tweets':'yes','phd':'yes'},True),
    
        ({'level':'Mid','lang':'C#','tweets':'no','phd':'no'},  True),
        ({'level':'Mid','lang':'C#','tweets':'no','phd':'yes'}, True),
        ({'level':'Mid','lang':'C#','tweets':'yes','phd':'no'}, True),
        ({'level':'Mid','lang':'C#','tweets':'yes','phd':'yes'},True),
    
        ##Junior
        ({'level':'Junior','lang':'Java','tweets':'no','phd':'no'},   False),
        ({'level':'Junior','lang':'Java','tweets':'no','phd':'yes'},  True),
        ({'level':'Junior','lang':'Java','tweets':'yes','phd':'no'},   False),
        ({'level':'Junior','lang':'Java','tweets':'yes','phd':'yes'},  False),
    
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'no'},  False),
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'yes'}, True),
        ({'level':'Junior','lang':'Python','tweets':'yes','phd':'no'}, False),
        ({'level':'Junior','lang':'Python','tweets':'yes','phd':'yes'},True),
    
        ({'level':'Junior','lang':'R','tweets':'no','phd':'no'},  False),
        ({'level':'Junior','lang':'R','tweets':'no','phd':'yes'}, True),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'no'}, False),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'yes'},False),
    
        ({'level':'Junior','lang':'C#','tweets':'no','phd':'no'},  True),
        ({'level':'Junior','lang':'C#','tweets':'no','phd':'yes'}, True),
        ({'level':'Junior','lang':'C#','tweets':'yes','phd':'no'}, False),
        ({'level':'Junior','lang':'C#','tweets':'yes','phd':'yes'},True),
    
        ##WithoutExp
        ({'level':'WithoutExp','lang':'Java','tweets':'yes','phd':'no'}, False),
        ({'level':'WithoutExp','lang':'C#','tweets':'yes','phd':'no'}, False),
    ]

In [46]:
tree = build_tree_id3(inputs50)
tree

('phd',
 {'no': ('level',
   {'Senior': ('lang',
     {'Java': ('tweets', {'no': False, 'yes': True, None: False}),
      'Python': ('tweets', {'no': False, 'yes': True, None: False}),
      'R': ('tweets', {'no': True, 'yes': False, None: False}),
      'C#': False,
      None: False}),
    'Mid': ('lang',
     {'Java': ('tweets', {'no': False, 'yes': True, None: False}),
      'Python': False,
      'R': ('tweets', {'no': True, 'yes': False, None: False}),
      'C#': True,
      None: False}),
    'Junior': ('lang',
     {'Java': False,
      'Python': False,
      'R': False,
      'C#': ('tweets', {'no': True, 'yes': False, None: False}),
      None: False}),
    'WithoutExp': False,
    None: False}),
  'yes': ('level',
   {'Senior': ('tweets', {'no': False, 'yes': True, None: False}),
    'Mid': True,
    'Junior': ('lang',
     {'Java': ('tweets', {'no': True, 'yes': False, None: False}),
      'Python': True,
      'R': ('tweets', {'no': True, 'yes': False, None: False}),
    